# Portfolio Construction
Construst portfolio based on LSTM online prediction results and backtest

In [ ]:
# Import all necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math


stockreturns = pd.read_csv('price.csv',index_col=0)
tickers = stockreturns.columns
lags = 20
for i in range(len(tickers)):
    ticker = tickers[i]
    stockreturns[ticker] = stockreturns[ticker].shift(-lags) / stockreturns[ticker]  - 1
stockreturns.dropna(inplace=True)
stockreturns.head()

In [ ]:
# Assume we have the predicted results
predicted = pd.read_csv('testPredict.csv',index_col=0)
predicted.head()

In [ ]:
balance_dates = []
portfolio_returns = []

n_stocks = 10  # long top 10 stocks in winning group and short top 10 stocks in losing group
holding_period = 20  # hold portfolio for 20 period

i = 0
while i < len(predicted):
    # Loc out the probalitily at t and rank
    probabilities = predicted.iloc[i,:]

    date = predicted.iloc[i,:].name
    ranked = pd.DataFrame(probabilities.rank(ascending = False))
    ranked.reset_index(inplace=True)
    ranked.columns = ['Company','Rank']
    ranked.set_index('Rank',inplace=True)
    ranked.sort_index(inplace=True)
    #print(ranked)
    
    # Select top 10 for winnning groups and lossing groups
    rankedstocks = ranked['Company'].values
    winningstocks = rankedstocks[:n_stocks]
    lossingstocks = rankedstocks[-n_stocks:]

    # Loc out the returns of selected stocks, average it and get the portfolio return for time t
    selected_return = []
    for j in range(n_stocks):
        selected_return.append(stockreturns.loc[date,winningstocks[j]])
        selected_return.append(-stockreturns.loc[date, lossingstocks[j]]) # "-" means returns for short position
    portfolio_return = np.average(selected_return)
    balance_dates.append(date)
    portfolio_returns.append(portfolio_return)
    i += holding_period

portfolio = pd.DataFrame(data={'Date':balance_dates,'Return':portfolio_returns})
portfolio.set_index('Date',inplace=True)
portfolio.index = pd.to_datetime(portfolio.index)
portfolio.head()

In [ ]:
portfolio['CumWealth'] = np.cumprod(pd.Series(1+portfolio['Return']))
# Plot the Cumulated return. I could also plot the enter and exit point in the residual plot later
portfolio['CumWealth'].plot()
plt.xlabel('Date')
plt.ylabel('Wealth')
plt.title('Cumulated Wealth of LSTM')

In [ ]:
# Calculate the yearly maximum drawdown
window = 252
Roll_Max = pd.rolling_max(portfolio['CumWealth'], window, min_periods=1)
Daily_Drawdown = portfolio['CumWealth'] / Roll_Max - 1
Max_Daily_Drawdown = abs(pd.rolling_min(Daily_Drawdown, window, min_periods=1))
MDD = np.max(Max_Daily_Drawdown)


#Calculate the sharpe ratio
sharpe = math.sqrt(12) * portfolio['Return'].mean() / portfolio['Return'].std()

#Print out the result
print('Cumulated Wealth: ',portfolio['CumWealth'].values[-1])
print('Maximum Drawdown: ',MDD)
print('Sharp Ratio:', sharpe)